# ATTN: This script should be run AFTER all embeddings are completed.


### START HERE IF SOURCING FROM df_manual_FOR_TRAINNING
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


# Descriptives and visualization


In [1]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [2]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


Using MPS


0it [00:00, ?it/s]

<Figure size 640x480 with 0 Axes>

In [3]:
def get_df_info(df, ivs_all=None):
    if ivs_all is None:
        ivs_all = [
            'Gender',
            'Gender_Num',
            'Gender_Female',
            'Gender_Mixed',
            'Gender_Male',
            'Gender_Female_n',
            'Gender_Male_n',
            'Gender_Female_% per Sector',
            'Gender_Male_% per Sector',
            'Age',
            'Age_Num',
            'Age_Older',
            'Age_Mixed',
            'Age_Younger',
            'Age_Older_n',
            'Age_Younger_n',
            'Age_Older_% per Sector',
            'Age_Younger_% per Sector',
        ]
    # Print Info
    print('\nDF INFO:\n')
    df.info()

    for iv in ivs_all:
        try:
            print('='*20)
            print(f'{iv}:')
            print('-'*20)
            if len(df[iv].value_counts()) < 5:
                print(f'{iv} Counts:\n{df[iv].value_counts()}')
                print('-'*20)
                print(f'{iv} Percentages:\n{df[iv].value_counts(normalize=True).mul(100).round(1).astype(float)}')
                print('-'*20)
            print(f'Min {iv} value: {df[iv].min()}')
            print(f'Max {iv} value: {df[iv].max()}')
            with contextlib.suppress(Exception):
                print('-'*20)
                print(f'{iv} Mean: {df[iv].mean().round(2).astype(float)}')
                print('-'*20)
                print(f'{iv} Standard Deviation: {df[iv].std().round(2).astype(float)}')
        except Exception:
            print(f'{iv} not available.')

    print('\n')


In [4]:
# Function to order categories
def categorize_df_gender_age(df, gender_order=None, age_order=None, ivs=None):
    if gender_order is None:
        gender_order = ['Female', 'Mixed Gender', 'Male']
    if age_order is None:
        age_order = ['Older', 'Mixed Age', 'Younger']
    if ivs is None:
        ivs = ['Gender', 'Age']
    # Arrange Categories
    for iv in ivs:
        if iv == 'Gender':
            order = gender_order
        elif iv == 'Age':
            order = age_order
        with contextlib.suppress(ValueError):
            df[iv] = df[iv].astype('category').cat.reorder_categories(order, ordered=True)

            df[iv] = pd.Categorical(
                df[iv], categories=order, ordered=True
            )
            df[f'{iv}_Num'] = pd.to_numeric(df[iv].cat.codes).astype('int64')

    return df


In [5]:
with open(f'{data_dir}df_manual_len.txt', 'r') as f:
    df_manual_len = int(f.read())

df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_trainning.pkl')
assert len(df_manual) == df_manual_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_manual_len} BUT IS OF LENGTH {len(df_manual)}'
print(f'Dataframe loaded with shape: {df_manual.shape}')


Dataframe loaded with shape: (5978, 58)


In [6]:
with open(f'{data_dir}df_jobs_len.txt', 'r') as f:
    df_jobs_len = int(f.read())

df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_for_classification.pkl')
assert len(df_jobs) == df_jobs_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_jobs_len} BUT IS OF LENGTH {len(df_manual)}'
print(f'Dataframe loaded with shape: {df_jobs.shape}')


Dataframe loaded with shape: (194820, 69)


In [7]:
dataframes = {
    'df_manual': df_manual,
    'df_jobs': df_jobs,
}

In [8]:
# All info
analysis_columns = [
    'Warmth',
    'Competence'
]

for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    df = categorize_df_gender_age(df)

    df.info()


++++++++++++++++++++ DF_MANUAL ++++++++++++++++++++
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5978 entries, 0 to 5977
Data columns (total 58 columns):
 #   Column                                            Non-Null Count  Dtype   
---  ------                                            --------------  -----   
 0   Job ID                                            5978 non-null   object  
 1   Job Description spacy_sentencized                 5978 non-null   object  
 2   Warmth                                            5978 non-null   int64   
 3   Competence                                        5978 non-null   int64   
 4   Dutch Requirement                                 5978 non-null   object  
 5   English Requirement                               5978 non-null   object  
 6   Search Keyword                                    5978 non-null   object  
 7   Platform                                          5978 non-null   object  
 8   Job Title                           

In [9]:
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    print('='*30)
    print('Sector info at Sentence Level')
    print('-'*30)
    get_df_info(df, ivs_all=['% Sector per Workforce'])
    print('-'*30)

++++++++++++++++++++ DF_MANUAL ++++++++++++++++++++
Sector info at Sentence Level
------------------------------

DF INFO:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5978 entries, 0 to 5977
Data columns (total 58 columns):
 #   Column                                            Non-Null Count  Dtype   
---  ------                                            --------------  -----   
 0   Job ID                                            5978 non-null   object  
 1   Job Description spacy_sentencized                 5978 non-null   object  
 2   Warmth                                            5978 non-null   int64   
 3   Competence                                        5978 non-null   int64   
 4   Dutch Requirement                                 5978 non-null   object  
 5   English Requirement                               5978 non-null   object  
 6   Search Keyword                                    5978 non-null   object  
 7   Platform                                    

In [15]:
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    min_sector = df['Sector'].loc[df['% Sector per Workforce'] == df['% Sector per Workforce'].min()].values[0]
    max_sector = df['Sector'].loc[df['% Sector per Workforce'] == df['% Sector per Workforce'].max()].values[0]
    print(f'"% Sector per Workforce":\nMin Sector: {df["% Sector per Workforce"].min():.1f}% in {min_sector}\nMax Sector: {df["% Sector per Workforce"].max():.1f}% in {max_sector}')
    print('-'*20)

++++++++++++++++++++ DF_MANUAL ++++++++++++++++++++
"% Sector per Workforce":
Min Sector: 0.1% in Energy supply
Max Sector: 31.4% in Commercial services
--------------------
++++++++++++++++++++ DF_JOBS ++++++++++++++++++++
"% Sector per Workforce":
Min Sector: 0.1% in Energy supply
Max Sector: 31.4% in Commercial services
--------------------


In [11]:
# Gender and Age info by sentence
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    print('='*30)
    print('Gender and Age info at Sentence Level')
    print('-'*30)
    get_df_info(df, ivs_all=ivs_all)
    print('-'*30)


++++++++++++++++++++ DF_MANUAL ++++++++++++++++++++
Gender and Age info at Sentence Level
------------------------------

DF INFO:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5978 entries, 0 to 5977
Data columns (total 58 columns):
 #   Column                                            Non-Null Count  Dtype   
---  ------                                            --------------  -----   
 0   Job ID                                            5978 non-null   object  
 1   Job Description spacy_sentencized                 5978 non-null   object  
 2   Warmth                                            5978 non-null   int64   
 3   Competence                                        5978 non-null   int64   
 4   Dutch Requirement                                 5978 non-null   object  
 5   English Requirement                               5978 non-null   object  
 6   Search Keyword                                    5978 non-null   object  
 7   Platform                            

In [12]:
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    for iv_perc in ivs_perc:
        min_sector = df['Sector'].loc[df[iv_perc] == df[iv_perc].min()].values[0]
        max_sector = df['Sector'].loc[df[iv_perc] == df[iv_perc].max()].values[0]
        print(f'{iv_perc}:\nMin Sector: {df[iv_perc].min():.1f}% in {min_sector}\nMax Sector: {df[iv_perc].max():.1f}% in {max_sector}')
        print('-'*20)


++++++++++++++++++++ DF_MANUAL ++++++++++++++++++++
Gender_Female_% per Sector:
Min Sector: 19.4% in Water supply and waste management
Max Sector: 84.3% in Health and social work activities
--------------------
Gender_Male_% per Sector:
Min Sector: 15.6% in Health and social work activities
Max Sector: 80.6% in Water supply and waste management
--------------------
Age_Older_% per Sector:
Min Sector: 18.9% in Accommodation and food serving
Max Sector: 58.3% in Water supply and waste management
--------------------
Age_Younger_% per Sector:
Min Sector: 44.4% in Water supply and waste management
Max Sector: 80.8% in Accommodation and food serving
--------------------
++++++++++++++++++++ DF_JOBS ++++++++++++++++++++
Gender_Female_% per Sector:
Min Sector: 12.5% in Construction
Max Sector: 84.3% in Health and social work activities
--------------------
Gender_Male_% per Sector:
Min Sector: 15.6% in Health and social work activities
Max Sector: 87.5% in Construction
--------------------
Ag

In [13]:
# Gender and Age info by job ad
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    print('='*30)
    print('Gender and Age info at Job Advertisement Level')
    print('-'*30)
    get_df_info(df.groupby(['Job ID']).first(), ivs_all=ivs_all)
    print('-'*30)


++++++++++++++++++++ DF_MANUAL ++++++++++++++++++++
Gender and Age info at Job Advertisement Level
------------------------------

DF INFO:

<class 'pandas.core.frame.DataFrame'>
Index: 117 entries, 3768944208 to pj_a4ac3e531abef752
Data columns (total 57 columns):
 #   Column                                            Non-Null Count  Dtype   
---  ------                                            --------------  -----   
 0   Job Description spacy_sentencized                 117 non-null    object  
 1   Warmth                                            117 non-null    int64   
 2   Competence                                        117 non-null    int64   
 3   Dutch Requirement                                 117 non-null    object  
 4   English Requirement                               117 non-null    object  
 5   Search Keyword                                    117 non-null    object  
 6   Platform                                          117 non-null    object  
 7   Job Title

In [ ]:
# Warmth and Competence percentages info by sentence
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    if df_name == 'df_jobs':
        dvs = dvs_prob

    print('='*30)
    print('Warmth and Competence info at Sentence Level')
    print('-'*30)
    get_df_info(df, ivs_all=dvs)
    print('-'*30)


In [ ]:
# Warmth and Competence info by job ad
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    if df_name == 'df_jobs':
        dvs = dvs_prob

    print('='*30)
    print('Warmth and Competence info at Job Advertisement Level')
    print('-'*30)
    get_df_info(df.groupby(['Job ID']).first(), ivs_all=dvs)
    print('-'*30)


In [ ]:
# Control variables info by sentence
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    print('='*30)
    print('Control varibales info at Sentence Level')
    print('-'*30)
    get_df_info(df, ivs_all = controls)
    print('-'*30)


In [ ]:
# Control variables info by job ad
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    print('='*30)
    print('Control varibales info at Job Advertisement Level')
    print('-'*30)
    get_df_info(df.groupby(['Job ID']).first(), ivs_all = controls)
    print('-'*30)


In [ ]:
# Imbalance Ratio
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    warmth_imbalance_ratio = df['Warmth'].loc[
        df['Warmth'] == 1].count()/df['Warmth'].loc[df['Warmth'] == 0
    ].count()
    competence_imbalance_ratio = df['Competence'].loc[
        df['Competence'] == 1].count()/df['Competence'].loc[df['Competence'] == 0
    ].count()

    all_imbalance_ratio_dict = {
        'Warmth': warmth_imbalance_ratio,
        'Competence': competence_imbalance_ratio
    }

    print('='*20)
    print('Imabalance Ratios')
    print('-'*10)
    print(f'Warmth IR: {warmth_imbalance_ratio:.2f}')
    print(f'Competence IR: {competence_imbalance_ratio:.2f}')
    print('='*20)


In [ ]:
# Ploting Warmth and Competence
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    dfs_to_transpose = [
        df['Warmth'].value_counts(normalize=True).mul(100).round(2).astype(float).to_frame().T,
        df['Competence'].value_counts(normalize=True).mul(100).round(2).astype(float).to_frame().T,
    ]
    if df_name == 'df_jobs':
        dfs_to_transpose.append(
            df['Warmth_Probability'].value_counts(normalize=True).mul(100).round(2).astype(float).to_frame().T,
        )
        dfs_to_transpose.append(
            df['Competence_Probability'].value_counts(normalize=True).mul(100).round(2).astype(float).to_frame().T,
        )

    df_warm_comp_transposed = pd.concat(dfs_to_transpose)

    fig, ax = plt.subplots()
    ax.set_title('Training Dataset: Warmth and Competence Sentence Percentages')
    ax.set_xlabel('Manually Annotated Sentencence Warmth and Competence Percentage from Total')

    df_warm_comp_transposed.plot(
        kind='barh', legend=True, stacked=True, ax=ax, color=['C5', 'C0'],
    )
    ax.legend(['Absent', 'Present'])

    for container in ax.containers:
        labels = [f'{width:.1f}%' for v in container if float(width:= v.get_width())]
        ax.bar_label(container, labels=labels, label_type='center', color='white', weight='bold')
        ax.set_xlabel('% Job Ad Sentences')

    for i, tick_label in enumerate(ax.get_ymajorticklabels()):
        ax.annotate(
            f'IR for {tick_label.get_text()} = {all_imbalance_ratio_dict[tick_label.get_text()]:.2f}',
            xy=(48, 0.3+i), ha='center', va='center'
        )

    for image_save_format in ['eps', 'png', 'svg']:
        fig.savefig(
            f'{plot_save_path}{df_name} Manually Annotated Warmth and Competence Sentences.{image_save_format}',
            format=image_save_format, dpi=3000, bbox_inches='tight'
        )


In [ ]:
# Ploting Gender and Age
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    df_gender_transposed = df['Gender'].value_counts(normalize=True).mul(100).round(2).astype(float).to_frame().T
    df_age_transposed = df['Age'].value_counts(normalize=True).mul(100).round(2).astype(float).to_frame().T

    fig, axs = plt.subplots(1, 2)
    fig.suptitle('Training Dataset: Gender and Age Sentence Percentages')

    df_gender_transposed.plot(
        kind='bar', legend=True, stacked=True, ax=axs[0], color=['C2', 'C0', 'C5']
    )
    df_age_transposed.plot(
        kind='bar', legend=True, stacked=True, ax=axs[1], color=['C0', 'C2', 'C5']
    )

    for ax in axs:
        for container in ax.containers:
            labels = [f'{height:.1f}%' for v in container if float(height:= v.get_height())]
            ax.bar_label(container, labels=labels, label_type='center', color='white', weight='bold')
            ax.set_ylabel('% Job Ad Sentences')
            handles, labels = ax.get_legend_handles_labels()
            order = [2, 1, 0] if 'Older' in labels else [2, 0, 1]
            ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order], loc='upper right', fontsize=8, )

    for image_save_format in ['eps', 'png', 'svg']:
        fig.savefig(
            f'{plot_save_path}{df_name} Manually Annotated Gender and Age Sentences.{image_save_format}',
            format=image_save_format, dpi=3000, bbox_inches='tight'
        )


In [ ]:
# Make stacked barplots with percentages
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    for iv in tqdm.tqdm(['Gender', 'Age', 'Sector']):
        fig, ax = plt.subplots()
        fig.suptitle(f'Training Dataset: Warmth and Competence Sentence Percentages per {iv}')

        vars_list = [iv, *dvs]
        df_pivot = df[
            vars_list
        ].pivot_table(
            index=iv, values=dvs, fill_value=0, aggfunc=lambda x: (100*x.sum())/len(df)
        )

        df_pivot.sort_values(by=iv, ascending=False).plot(kind='barh', legend=True, stacked=True, ax=ax, color=['C0', 'C5'])

        ax.legend(dvs)

        for container in ax.containers:
            labels = [f'{width:.1f}%' for v in container if float(width:= v.get_width())]
            if iv != 'Sector':
                ax.bar_label(container, labels=labels, label_type='center', color='white', fontsize=8)
            ax.set_xlabel('% Job Ad Sentences')
            ax.set_ylabel(iv)
            ax.legend(loc='upper right', fontsize=8)

        for image_save_format in ['eps', 'png', 'svg']:
            fig.savefig(
                f'{plot_save_path}{df_name} Barplot - Manually Annotated {iv} x {dvs[0]} and {dvs[1]} Sentences.{image_save_format}',
                format=image_save_format, dpi=3000, bbox_inches='tight'
            )


In [ ]:
def make_2ax_lineplot(ivs, dvs):
    line_plot = sns.lineplot(
        data=df.groupby(ivs[0])[dvs[0]].agg('mean').astype(float), marker='o', legend='full', ax=axs[0], color='C5'
    )
    line_plot = sns.lineplot(
        data=df.groupby(ivs[0])[dvs[1]].agg('mean').astype(float), marker='o', legend='full', ax=axs[0], color='C0'
    )
    line_plot = sns.lineplot(
        data=df.groupby(ivs[1])[dvs[0]].agg('mean').astype(float), marker='o', legend='full', ax=axs[1], color='C5'
    )
    line_plot = sns.lineplot(
        data=df.groupby(ivs[1])[dvs[1]].agg('mean').astype(float), marker='o', legend='full', ax=axs[1], color='C0'
    )

    fig.legend(dvs, fontsize=10)

    return line_plot


In [ ]:
# Make line plots 2 ax
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    if df_name == 'df_jobs':
        dvs = dvs_prob

    fig, axs = plt.subplots(1, 2)
    fig.suptitle('Warmth and Competence x Gender and Age')

    line_plot = make_2ax_lineplot(ivs, dvs)

    # line_plot = sns.lineplot(
    #     data=df.groupby('Gender')['Warmth'].agg('mean').astype(float), marker='o', legend='full', ax=axs[0], color='C5'
    # )
    # line_plot = sns.lineplot(
    #     data=df.groupby('Gender')['Competence'].agg('mean').astype(float), marker='o', legend='full', ax=axs[0], color='C0'
    # )
    # line_plot = sns.lineplot(
    #     data=df.groupby('Age')['Warmth'].agg('mean').astype(float), marker='o', legend='full', ax=axs[1], color='C5'
    # )
    # line_plot = sns.lineplot(
    #     data=df.groupby('Age')['Competence'].agg('mean').astype(float), marker='o', legend='full', ax=axs[1], color='C0'
    # )

    # fig.legend(dvs, fontsize=10)

    fig = line_plot.get_figure()

    for image_save_format in ['eps', 'png', 'svg']:
        fig.savefig(
            f'{plot_save_path}{df_name} Line Plot (2 ax) - Manually Annotated Sentences.{image_save_format}',
            format=image_save_format, dpi=3000, bbox_inches='tight'
        )
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()

In [ ]:
def make_4ax_lineplot(ivs, dvs):
    line_plot = sns.lineplot(
        data=df.groupby(ivs[0])['Warmth'].agg('mean').astype(float), marker='o', legend='full', ax=axs[0, 0], color='C5'
    )
    line_plot = sns.lineplot(
        data=df.groupby(ivs[0])['Competence'].agg('mean').astype(float), marker='o', legend='full', ax=axs[0, 1], color='C0'
    )
    line_plot = sns.lineplot(
        data=df.groupby(ivs[1])['Warmth'].agg('mean').astype(float), marker='o', legend='full', ax=axs[1, 0], color='C5'
    )
    line_plot = sns.lineplot(
        data=df.groupby(ivs[1])['Competence'].agg('mean').astype(float), marker='o', legend='full', ax=axs[1, 1], color='C0'
    )

    fig.legend(dvs, fontsize=10)

    return line_plot

In [ ]:
# Make line plots 4 ax
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    if df_name == 'df_jobs':
        dvs = dvs_prob

    fig, axs = plt.subplots(2, 2)
    fig.suptitle('Warmth and Competence x Gender and Age')

    line_plot = make_4ax_lineplot(ivs, dvs)

    # line_plot = sns.lineplot(
    #     data=df.groupby('Gender')['Warmth'].agg('mean').astype(float), marker='o', legend='full', ax=axs[0, 0], color='C5'
    # )
    # line_plot = sns.lineplot(
    #     data=df.groupby('Gender')['Competence'].agg('mean').astype(float), marker='o', legend='full', ax=axs[0, 1], color='C0'
    # )
    # line_plot = sns.lineplot(
    #     data=df.groupby('Age')['Warmth'].agg('mean').astype(float), marker='o', legend='full', ax=axs[1, 0], color='C5'
    # )
    # line_plot = sns.lineplot(
    #     data=df.groupby('Age')['Competence'].agg('mean').astype(float), marker='o', legend='full', ax=axs[1, 1], color='C0'
    # )

    # fig.legend(dvs, fontsize=10)

    fig = line_plot.get_figure()

    for image_save_format in ['eps', 'png', 'svg']:
        fig.savefig(
            f'{plot_save_path}{df_name} Line Plot (4 ax) - Manually Annotated Sentences.{image_save_format}',
            format=image_save_format, dpi=3000, bbox_inches='tight'
        )
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()

In [ ]:
def make_violin_plot(ivs, dvs):
    violin_plot = sns.violinplot(
        data=df, x='Gender', y='Warmth', palette=['C5', 'C2', 'C0'], inner='quartile', ax=axs[0, 0]
    )
    violin_plot = sns.violinplot(
        data=df, x='Gender', y='Competence', palette=['C5', 'C2', 'C0'], inner='quartile', ax=axs[0, 1]
    )
    violin_plot = sns.violinplot(
        data=df, x='Age', y='Warmth', palette=['C5', 'C2', 'C0'], inner='quartile', ax=axs[1, 0]
    )
    violin_plot = sns.violinplot(
        data=df, x='Age', y='Competence', palette=['C5', 'C2', 'C0'], inner='quartile', ax=axs[1, 1]
    )

    return violin_plot

In [ ]:
# Make violin plots
for df_name, df in dataframes.items():
    print(f'{"+"*20} {df_name.upper()} {"+"*20}')

    if df_name == 'df_jobs':
        dvs = dvs_prob

    fig, axs = plt.subplots(2, 2)
    fig.suptitle('Distribution of Warmth and Competence Scores')

    violin_plot = make_violin_plot(ivs, dvs)

    # violin_plot = sns.violinplot(
    #     data=df, x='Gender', y='Warmth', palette=['C5', 'C0'], inner='quartile', color='C0', ax=axs[0, 0]
    # )
    # violin_plot = sns.violinplot(
    #     data=df, x='Gender', y='Competence', palette=['C5', 'C0'], inner='quartile', color='C0', ax=axs[0, 1]
    # )
    # violin_plot = sns.violinplot(
    #     data=df, x='Age', y='Warmth', palette=['C5', 'C0'], inner='quartile', color='C0', ax=axs[1, 0]
    # )
    # violin_plot = sns.violinplot(
    #     data=df, x='Age', y='Competence', palette=['C5', 'C0'], inner='quartile', color='C0', ax=axs[1, 1]
    # )

    fig = violin_plot.get_figure()

    for image_save_format in ['eps', 'png', 'svg']:
        fig.savefig(
                f'{plot_save_path}{df_name} Violin Plot - Manually Annotated Sentences.{image_save_format}',
                format=image_save_format, dpi=3000, bbox_inches='tight'
            )


# ATTN: This script should be run AFTER all visualizations are completed.


### START HERE IF SOURCING FROM df_manual_FOR_TRAINNING
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


# Make descriptive tables


In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
# Function to order categories
def categorize_df_gender_age(df, gender_order=None, age_order=None, ivs=None):
    if gender_order is None:
        gender_order = ['Female', 'Mixed Gender', 'Male']
    if age_order is None:
        age_order = ['Older', 'Mixed Age', 'Younger']
    if ivs is None:
        ivs = ['Gender', 'Age']
    # Arrange Categories
    for iv in ivs:
        if iv == 'Gender':
            order = gender_order
        elif iv == 'Age':
            order = age_order
        with contextlib.suppress(ValueError):
            df[iv] = df[iv].astype('category').cat.reorder_categories(order, ordered=True)

            df[iv] = pd.Categorical(
                df[iv], categories=order, ordered=True
            )
            df[f'{iv}_Num'] = pd.to_numeric(df[iv].cat.codes).astype('int64')

    return df


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_trainning.pkl')


### Gender and Age tables

In [ ]:
df = df_manual.copy()
gender_order = ['Female', 'Male', 'Mixed Gender']
age_order = ['Older', 'Younger', 'Mixed Age']

ivs_dict = {'Gender': gender_order, 'Age': age_order}

df_desc_gender_dict = defaultdict(list)
df_desc_age_dict = defaultdict(list)

variables = ['Gender', 'Age']
levels = ['Job Advertisement', 'Sentence']
data_types = ['Manually Annotated', 'Collected']

for v, level, data_type in tqdm_product(
    variables, levels, data_types
):
    df = categorize_df_gender_age(df)

    if level.title() == 'Job Advertisement':
        level_df = df.groupby(['Job ID']).first()
    elif level.title() == 'Sentence':
        level_df = df

In [ ]:
# Function to make descriptives tables
def genage_make_descriptives_table(df, v, level, gender_order=None, age_order=None):

    if gender_order is None:
        gender_order = ['Female', 'Male', 'Mixed Gender']
    if age_order is None:
        age_order = ['Older', 'Younger', 'Mixed Age']
    
    ivs_dict = {'Gender': gender_order, 'Age': age_order}

    df = categorize_df_gender_age(df)

    if level.title() == 'Job Advertisement':
        level_df = df.groupby(['Job ID']).first()
    elif level.title() == 'Sentence':
        level_df = df
    else:
        raise AssertionError(f'Specified level {level} not in data.')

    if v in list(ivs_dict.keys()):
        cat_dict = ivs_dict
        index = [
            f'{v_cat}-dominated'
            if 'Mixed' not in v_cat
            else
            f'{"-".join(v_cat.split())}'
            for v_cat in cat_dict[v]
        ]
        caption = [
            f'{v}_{v_cat.split()[0]}'
            for v_cat in cat_dict[v]
        ]

    desc_dict = {
        'Sectors': index,
        'n': [
            level_df[v].value_counts()[v_cat]
            for v_cat in cat_dict[v]
        ],
        '%': [
            level_df[v].value_counts(normalize=True).mul(100).round(2).astype(float)[v_cat]
            for v_cat in cat_dict[v]
        ],
        'M': [
            level_df[caption].mean().round(2).astype(float)[i]
            for i in range(len(cat_dict[v]))
        ],
        'S.D.': [
            level_df[caption].std().round(2).astype(float)[i]
            for i in range(len(cat_dict[v]))
        ]
    }

    # Make DF from dict
    df_desc = pd.DataFrame(desc_dict)
    df_desc = df_desc.set_index('Sectors')

    return df_desc


In [ ]:
df_desc_gender_dict = defaultdict(list)
df_desc_age_dict = defaultdict(list)

variables = ['Gender', 'Age']
levels = ['Job Advertisement', 'Sentence']
data_types = ['Manually Annotated', 'Collected']

for v, level, data_type in tqdm_product(
    variables, levels, data_types
):
    cols = [
        (f'{data_type.title()} Job Advertisements', v, level.title(), 'n'),
        (f'{data_type.title()} Job Advertisements', v, level.title(), '%'),
        (f'{data_type.title()} Job Advertisements', v, level.title(), 'M'),
        (f'{data_type.title()} Job Advertisements', v, level.title(), 'S.D.'),
    ]

    df_desc_genage = genage_make_descriptives_table(df=df_manual, v=v, level=level)

    df_desc_genage.columns = pd.MultiIndex.from_tuples(cols)

    if v == 'Gender':
        df_desc_gender_dict[data_type].append(df_desc_genage)
    elif v == 'Age':
        df_desc_age_dict[data_type].append(df_desc_genage)


In [ ]:
df_desc_genage_dict = defaultdict(list)

for df, data_type in tqdm_product(
    [df_desc_gender_dict, df_desc_age_dict],
    data_types,
):
    category = df[data_type][0].columns.get_level_values(level=1)[0]
    if df[data_type][0].columns.get_level_values(level=2).str.contains('Job Advertisement').all():
        df_desc_genage_dict[category].append(
            pd.concat(
                [
                    df[data_type][0], df[data_type][1]
                ],
                axis='columns'
            )
        )
    else:
        df_desc_genage_dict[category].append(
            pd.concat(
                [
                    df[data_type][1], df[data_type][0]
                ],
                axis='columns'
            )
        )


In [ ]:
for cat, df in df_desc_genage_dict.items():
    df = pd.concat([df[0], df[1]], axis='columns')

    if cat == 'Gender':
        df_desc_gender = df
    elif cat == 'Age':
        df_desc_age = df


In [ ]:
# Save Tables
# Gender
df_desc_gender.to_csv(f'{table_save_path}Gender - Job Advertisement Descriptives.csv', index=True)
df_desc_gender.to_pickle(f'{table_save_path}Gender - Job Advertisement Descriptives.pkl')
with pd.option_context('max_colwidth', 10000000000):
    df_desc_gender.to_latex(f'{table_save_path}Gender - Job Advertisement Descriptives.tex', index=True, longtable=True, escape=True, multicolumn=True, multicolumn_format='c', position='H', caption='Number and proportion of job advertisements sample, and resulting sentences for gender and age homogeneous and heterogeneous sectors', label='Descriptives')
df_desc_gender.to_markdown(f'{table_save_path}Gender - Job Advertisement Descriptives.md', index=True)
# save_sector_excel(df_sectors_all, data_save_dir)

# Age
df_desc_age.to_csv(f'{table_save_path}Age - Job Advertisement Descriptives.csv', index=True)
df_desc_age.to_pickle(f'{table_save_path}Age - Job Advertisement Descriptives.pkl')
with pd.option_context('max_colwidth', 10000000000):
    df_desc_age.to_latex(f'{table_save_path}Age - Job Advertisement Descriptives.tex', index=True, longtable=True, escape=True, multicolumn=True, multicolumn_format='c', position='H', caption='Number and proportion of job advertisements sample, and resulting sentences for gender and age homogeneous and heterogeneous sectors', label='Descriptives')
df_desc_age.to_markdown(f'{table_save_path}Age - Job Advertisement Descriptives.md', index=True)
# save_sector_excel(df_sectors_all, data_save_dir)


### Warmth and Competence tables


In [ ]:
variables = ['Warmth', 'Competence']

cols = [
    ('Manually Annotated Job Advertisements', 'n'),
    ('Manually Annotated Job Advertisements', '%'),
    ('Manually Annotated Job Advertisements', 'M'),
    ('Manually Annotated Job Advertisements', 'S.D.'),
]

binary_order = [0, 1]
cat_dict = {'Warmth': binary_order, 'Competence': binary_order}

desc_warmcomp_dict = {
    'Frames': list(cat_dict.keys()),
    'n': [
        df_manual[v].value_counts()[1]
        for v in variables
    ],
    '%': [
        df_manual[v].value_counts(normalize=True).mul(100).round(2).astype(float)[1]
        for v in variables
    ],
    'M': list(
        df_manual[variables].mean().round(2).astype(float)
    ),
    'S.D.': list(
        df_manual[variables].std().round(2).astype(float)
    )
}
    
# Make df_manual from dict
df_desc_warmcomp = pd.DataFrame(desc_warmcomp_dict)
df_desc_warmcomp = df_desc_warmcomp.set_index('Frames')
df_desc_warmcomp.columns = pd.MultiIndex.from_tuples(cols)


In [ ]:
# Save Tables
df_desc_warmcomp.to_csv(f'{table_save_path}Warmth and Competence - Job Advertisement Descriptives.csv', index=True)
df_desc_warmcomp.to_pickle(f'{table_save_path}Warmth and Competence - Job Advertisement Descriptives.pkl')
with pd.option_context('max_colwidth', 10000000000):
    df_desc_warmcomp.to_latex(f'{table_save_path}Warmth and Competence - Job Advertisement Descriptives.tex', index=True, longtable=True, escape=True, multicolumn=True, multicolumn_format='c', position='H', caption='Number and proportion of job advertisements sample, and resulting sentences for gender and age homogeneous and heterogeneous sectors', label='Descriptives')
df_desc_warmcomp.to_markdown(f'{table_save_path}Warmth and Competence - Job Advertisement Descriptives.md', index=True)
# save_sector_excel(df_sectors_all, data_save_dir)
